# Étude sur les facteurs qui déterminent la santé des enfants aux États-Unis

# Importation des libraries 

In [ ]:
# -q : quiet --> réduction des messages
%pip install -q -r requirements.txt

In [ ]:
import s3fs # pour récupérer les fichiers de l'espace de stockage S3
import pandas as pd # pour la gestion des dataframe
import geopandas as gpd # pour la gestion des données géographiques (fichiers .shp)
import matplotlib.pyplot as plt

from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import numpy as np
import os
import prince # pour l'analyse en Composantes Multiples
import seaborn as sns
from matplotlib.patches import Ellipse
from scipy.stats import norm

import ipywidgets as widgets
from IPython.display import display

# Librairies propres pour faciliter la lecture du notebook

import script.clean_data as cd
import script.analyse_data as ad
import script.model as model
import folium # création de cartes intéractives
from folium.features import Choropleth

## Récupération des bases de données

In [ ]:
# Maximum 15 secondes

# (depuis le stockage S3 d'un membre du groupe via le dossier public diffusion)
# Stocker les fichiers dans un dossier data/nsch

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

chemin_lecture_nsch = "inacampan/diffusion/Determinants_of_children-s_health/NSCH/"
chemin_ecriture_nsch = "data/nsch/"

# Lecture des fichiers des bases de données
dfs = cd.lecture_fichier_sas(fs, chemin_lecture_nsch, chemin_ecriture_nsch)

# récuperér le guide technique du questionnaire
# ce fichier inclut le nom de la variable, la question qu'elle encode, l'année du questionnaire
# mais également des notes pour la compréhension de la base de données initiale

guide = cd.lecture_fichier_csv(fs, f"{chemin_lecture_nsch}NSCH_Dictionary.csv", f"{chemin_ecriture_nsch}NSCH_Dictionary.csv")

# Element de précaution : GUIDE contient plusieurs index de meme nom
# mais les caractéristiques ne sont pas identiques

In [ ]:
# Maximum 2 secondes

# Récuperer les fichiers associés au shapefile - utiles pour les cartes (depuis le stockage S3 d'un membre du groupe)

chemin_lecture_map = "inacampan/diffusion/Determinants_of_children-s_health/Map/"
chemin_ecriture_map = "data/map/"

# lecture du fichier
gdf = cd.lecture_fichier_shapefile(fs, chemin_lecture_map, chemin_ecriture_map)

In [ ]:
# Maximum 2 secondes
# Récuperer les données économiques - fichier csv
chemin_lecture_eco = "inacampan/diffusion/Determinants_of_children-s_health/Economic/"
chemin_ecriture_eco = "data/economic/"

# lecture du fichier
gdp = cd.lecture_fichier_csv(fs, 
                             f"{chemin_lecture_eco}SASUMMARY__ALL_AREAS_1998_2024.csv", 
                             f"{chemin_ecriture_eco}SASUMMARY__ALL_AREAS_1998_2024.csv")

## Description des données et statistiques descriptives

Conformément au site qui héberge les bases de données NSCH, le questionnaire de présélection (NSCH-S1) déterminait si l’adresse représentait une résidence occupée et s’il y avait des enfants admissibles âgés de 0 à 17 ans vivant à cette adresse échantillonnée.

Le questionnaire thématique ("topical") comprenait des questions détaillées portant sur un enfant sélectionné au hasard dans le ménage. Les ménages recevaient l’un des trois questionnaires thématiques spécifiques à l’âge, en fonction de l’âge de l’enfant échantillonné : 
* NSCH-T1 (ou T1) pour les enfants âgés de 0 à 5 ans,
* NSCH-T2 (ou T2) pour les enfants âgés de 6 à 11 ans, ou
* NSCH-T3 (ou T3) pour les enfants âgés de 12 à 17 ans.

Le type du questionnaire est codé dans la variable ```FORMTYPE```.
Dans notre projet nous regardons uniquement le questionnaire "topical". 

In [ ]:
df = dfs["2023"] # regardons d'abord l'année 2023
df.describe()

In [ ]:
# La base de données de 2023 est dense, avec un total de 452 de variables sur 55162 individus (à travers les états-Unis).
# Sur l'ensemble des enfants dans l'échantillons : 
# (i) 21524 sont âgés de 0 à 5 ans,
# (ii) 18397 sont âgés de 6 à 11 ans et 
# (iii) 15241 sont âgés de 11 à 17 ans.

print(df["FORMTYPE"].value_counts())
print(df["FIPSST"].value_counts()[:5])

On regarde le pourcentage de valeurs manquantes par colonne, par ordre décroissant.
On observe des taux de valeurs manquantes très élevés pour certaines questions qui ne sont posées que dans des cas très spécifiques et qui dépendent d’une réponse précédente. (Cela est rassurant.)

Par exemple, pour la variable ```CYSFIB_SCREEN``` (avec un taux de 99.94% de valeurs manquantes), celle-ci encode la question :

**Was this condition identified through a blood test done shortly after birth? … These tests are sometimes called newborn screening.** Cette question est ignorée si ```CYSTFIB = 2```. La variables ```CYSFIB``` a un taux très très faible (0.19%) de valeurs manquantes et parmi les valeurs prises, ```CYSFIB!=2``` dans seulement 30 questionnaires.

Pour les variables ```DIABETES_DESC``` et ```DIABETES_CURR```, les questions qu'elles encodent sont ignorées dès lorsque ```DIABETES = 2``` (ce qui regroupe seulement 40 questionnaires).

In [ ]:
missing_values = df.isna().mean().sort_values(ascending=False) * 100

print("Valeurs manquantes en pourcentage par colonne\n",
    missing_values)
print("Nombre de colonnes avec au moins 50% de valeurs manquantes\n",(missing_values >= 50).sum())

# Illustration de l'exemple de CYSTFIB et DIABETES
print("---------------------------------")
print("Informations sur la variable CYSTFIB_SCREEN : ", guide[guide["Variable"] == "CYSTFIB_SCREEN"]["Universe"].iloc[0], "\n",
    guide[guide["Variable"] == "CYSTFIB"]["Question"].iloc[0])
print("Valeurs manquantes pour la variable CYSTFIB : \n",
    (df["CYSTFIB"].isna().mean() * 100).round(2), "%")
print(df["CYSTFIB"].value_counts())

print("---------------------------------")
print("Informations sur la variable DIABETES_CURR : ", guide[guide["Variable"] == "DIABETES_CURR"]["Universe"].iloc[0], "\n",
    guide[guide["Variable"] == "DIABETES"]["Question"].iloc[0])
print("Valeurs manquantes pour la variable DIABETES : \n",
    (df["DIABETES"].isna().mean() * 100).round(2), "%")
print(df["DIABETES"].value_counts())

### Choix des variables d'intérêt

Dans le guide, les variables sont classées dans plusieurs grandes catégories, en fonction de la question de recherche et également de la base de données associée. En vue de notre base de données, on peut garder uniquement les variables de type "Topical" et "Operational" (colonne "Source").

In [ ]:
guide = guide[guide["Source"].isin({"Topical", "Operational"})]

years = {"2021", "2022", "2023", "2024"}

# On garde uniquement des variables qui sont communes aux quatre dataset NSCH
guide = guide[guide["Survey Years"].apply(
    lambda x: years <= set(y.strip() for y in str(x).split(","))
)]

# Cela réduit la base à 318 variables communes entre les databases
# Netoyer les dataframe NSCH avec cette contrainte, en rajoutant le code de l'état de résidance
# comme variable admissible, ainsi que la taille et le poids de l'enfant.

variables_com = set(guide["Variable"])
variables_com = variables_com | {"FIPSST", "WEIGHT", "HEIGHT"}

for year in ["2023", "2022", "2021", "2024"]:
    df = dfs[year]
    df = df.loc[:, df.columns.isin(variables_com)]
    dfs[year] = df

In [ ]:
counts = guide.groupby(["Source", "Topic"])["Variable"].count().reset_index(name="count")
print("Nombre de variable par groupe de (Source, Topic)\n",counts.to_string())

Listes de nos variables de santé générale :

- K2Q01 : General Health (1 = Excellent -> 5 = Poor)
- K2Q01_D : Teeth description (1 = Excellent -> 5 = Poor)
- K2Q40A : Asthma (1 = Yes, 2 = No)
- K2Q42A : Epilepsy (1 = Yes, 2 = No)
- K2Q43B : Deafness (1 = Yes, 2 = No)
- K2Q61A : Cerebral Palsy (1 = Yes, 2 = No)
- BLINDNESS : Blindness (1 = Yes, 2 = No)
- BLOOD : Blood disorder (1 = Yes, 2 = No)
- BREATHING : Difficulty Breathing (1 = Yes, 2 = No)
- CAVITIES : Decayed teeth or cavities (1 = Yes, 2 = No)
- CYSTFIB : Cystic fibrosis (1 = Yes, 2 = No)
- HEADACHE : Frequent or severe headache (1 = Yes, 2 = No)
- HEART : Heart condition (1 = Yes, 2 = No)
- STOMACH : Frequent or chronic difficulty with digesting food, including stomach/intestinal problems, constipation, or diarrhea (1 = Yes, 2 = No)
- TOOTHACHES : Frequent or chronic toothaches (1 = Yes, 2 = No)

In [ ]:
# Sélectionner des variables dans l'univers "All Children" 
# => cad des questions posées à toutes les groupes d'age

filter_variables = {var for var in variables_com
    if guide[guide["Variable"] == var]["Universe"].item() == "All Children"
}

health_category_vars = ["K2Q01", "K2Q01_D"]
health_bin_vars = ["K2Q40A", "K2Q42A", "K2Q43B", "K2Q61A", "BLINDNESS", 
                   "BLOOD", "BREATHING", "CAVITIES", "CYSTFIB", "HEADACHE", 
                   "HEART", "STOMACH", "TOOTHACHES"]
health_vars = health_category_vars + health_bin_vars

Listes de nos variables concernant des troubles/syndromes :

- K2Q30A : Learning Disability (1 = Yes, 2 = No)
- K2Q31A : ADD/ADHD (1 = Yes, 2 = No)
- K2Q32A : Depression (1 = Yes, 2 = No)
- K2Q33A : Anxiety (1 = Yes, 2 = No)
- K2Q34A : Behavioral problems (1 = Yes, 2 = No)
- K2Q35A : Autism (1 = Yes, 2 = No)
- K2Q36A : Developmental delay (1 = Yes, 2 = No)
- K2Q37A : Speech disorder (1 = Yes, 2 = No)
- K2Q38A : Tourette syndrome (1 = Yes, 2 = No)
- K2Q60A : Intellectual disability (1 = Yes, 2 = No)
- DOWNSYN : Down Syndrom (1 = Yes, 2 = No)
- HCABILITY : Health Affected Ability (1 = No health conditions, 2 = Never -> 5 = Always)

Listes de nos variables concernant l'environnement de l'enfant :

- ACE6 : Saw or heard adults Slap, Hit, Kick, Punch Others (1 = Yes, 2 = No)
- ACE7 : Victim of Violence (1 = Yes, 2 = No)
- ACE8 : Lived with anyone who was mentally ill, suicidal, or severely depressed (1 = Yes, 2 = No)
- ACE9 : Lived with anyone who had a problem with alcohol or drugs (1 = Yes, 2 = No)
- ACE10 : Treated Unfairly Because of Race (1 = Yes, 2 = No)
- ACE11 : Treated Unfairly Because of Health Condition (1 = Yes, 2 = No)

In [ ]:
mental_category_vars = ["HCABILITY"]
mental_bin_vars = ["K2Q30A", "K2Q31A", "K2Q32A", "K2Q33A", "K2Q34A", "K2Q35A", 
                   "K2Q36A", "K2Q37A", "K2Q38A", "K2Q60A", "DOWNSYN", "ACE6", 
                   "ACE7", "ACE8", "ACE9", "ACE10", "ACE11"]
mental_health_vars = mental_category_vars + mental_bin_vars

Listes de nos variables :

- CURRINS : Currently insured (1 = Yes, 2 = No)
- FOODSIT : Food situation at home (1 = "We could always afford to eat good nutritious meals." -> 4 = "Often we could not afford enough to eat.")
- ACE1 : Hard to Cover Basics Like Food or Housing (1 = "Never" -> 4 = "Very Often")

In [ ]:
NSCH_eco_bin_vars = ["CURRINS"]
NSCH_eco_cat_vars = ["FOODSIT", "ACE1"]
NSCH_eco_vars = NSCH_eco_bin_vars + NSCH_eco_cat_vars

In [ ]:
operational_vars = ["FIPSST", "FWC", "FORMTYPE", "WEIGHT", "HEIGHT"]

In [ ]:
# Proposition : 
final_variables = list(set(health_vars) | set(mental_health_vars) | 
                set(NSCH_eco_vars) | set(operational_vars))

topical_variables = list(set(health_vars) | set(mental_health_vars) | 
                set(NSCH_eco_vars))
     
guide = guide[
    guide["Variable"].isin(final_variables) &  # garde seulement si la variable est dans variables_finales
    guide["Survey Years"].apply(lambda x: "2024" in x)  # garde seulement si '2024' est dans years
]

cd.write_questions(final_variables, guide)

In [ ]:
for year in ["2024", "2023", "2022", "2021"]:
    df = dfs[year]
    df = df.loc[:, df.columns.isin(final_variables)]
    dfs[year] = df

In [ ]:
df = dfs["2023"]
df.head()

In [ ]:
missing_values = df.isna().mean().sort_values(ascending=False) * 100
print(missing_values)

In [ ]:
# Cas particulier de HEIGHT et WEIGHT
df_not_t1 = df[df['FORMTYPE'] != 'T1']

# Compter les NA dans la colonne 'height' et 'weight
nb_na_height = df_not_t1['HEIGHT'].isna().sum()
nb_na_weight = df_not_t1['WEIGHT'].isna().sum()

pct_na_height = (nb_na_height / len(df_not_t1)) * 100
pct_na_weight = (nb_na_weight / len(df_not_t1)) * 100

print(f"Pourcentage de HEIGHT manquants (pour la sous-population > 5 ans): {pct_na_height:.2f}%")
print(f"Pourcentage de WEIGHT manquants (pour la sous-population > 5 ans): {pct_na_weight:.2f}%")

In [ ]:
for var in df.columns:
    print("Variable : " + var + " prend " + str(len(df[var].unique())) + " valeurs uniques")
    print(df[var].unique())

# On a une variable qui est à moitié catégorielles - le ID de l'état
# On a trois variables continue
#    - la pondération de la réponse (FWC) : 0 valeurs manquantes
#     - le poids de l'enfant (seulement pour > 5 ans)
#     - la taille de l'enfant (seulement pour > 5 ans)

### Imputation

Pour s'y prendre, on utilise la librairie scikit-learn et une imputation via la méthode des K plus proches voisins : [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html). Nous utilisons un nombre de voisins égal à 3 (au lieu du 5 par défaut), par soucis de simplicité. De plus, nos variables manquantes étant catégorielles, un choix trop élevé de voisins à considérer n'est pas justifiable. Pour ```HEIGHT``` et ```WEIGHT``` on effectue une imputation uniquement au niveau de la sous-population telle que ```"FORMTYPE"!='T1'```.

In [ ]:
# Cette cellule prend environ 5 à 12 minutes pour s'executer
# On recommande d'executer directement la cellule suivante qui récupère ce résultat depuis le S3

"""
years = ["2024", "2023", "2022", "2021"]
dfs_final = cd.impute_values_over_dataset(years, dfs)
"""

In [ ]:
# Sauvegarde initiale sous S3 - via le compte inacampan - executé une seule fois 

# cd.write_on_S3(fs, years, dfs_final)


In [ ]:
dfs_final = cd.read_on_S3(fs, years)

In [ ]:
cd.test_imputed(years, dfs_final)

In [ ]:
# visualisation de bar plot pour chaque variable catégorielle, post imputation
# possibilité de sélectionner la variable et l'année suhaitée de la base de données

year_selector = widgets.Dropdown(
    options = ["2024", "2023", "2022", "2021"],
    description = 'Année',
    value = "2023"
)

var_selector = widgets.Dropdown(
    options=list(topical_variables),
    description='Variable:',    
    value='BREATHING'
)

interactive_plot = widgets.interactive(
    ad.bar_plot,
    variable=var_selector,
    year=year_selector,
    dfs=widgets.fixed(dfs_final),
    guide=widgets.fixed(guide)
)

display(interactive_plot)

### Réalisation d'une Analyse en Composantes Multiples

La MCA (Multiple Correspondence Analysis) est une technique d’analyse statistique qui permet de réduire la dimensionnalité de données catégorielles et de visualiser les relations entre variables et individus. La librairie [Prince](https://pypi.org/project/prince/) en Python facilite cette analyse et la visualisation des résultats.

In [ ]:
df_mca, mca = ad.mca_analysis("2023", dfs_final, 
                              ["FWC", "FIPSST", "FORMTYPE", "HEIGHT", "WEIGHT"])

In [ ]:
mca.eigenvalues_summary

In [ ]:
mca.row_coordinates(df_mca).head()

In [ ]:
mca.column_coordinates(df_mca).head()

On voit beaucoup de points regroupés vers le coin gauche-bas. Cela signifie que la majorité des individus partagent des modalités similaires sur les variables catégorielles.

Les points plus éloignés sont des individus atypiques ou ayant des combinaisons rares de modalités.

In [ ]:
plot_individus = ad.mca_plot_individuals(df_mca, mca)

In [ ]:
# Visualisation par groupes
# Le dataframe comporte beaucoup de colonnes => difficile de plotter un graphe à la fois

# Ajouter un coté interactif : l'utilisateur choisi la variable de regroupement qu'il souhaite

var_selector = widgets.Dropdown( # Dropdown : liste avec les options qui s'affiche en cliquant
    df_mca=widgets.fixed(df_mca),  # valeur fixée au préalable, l'utilisateur n'a pas de choix
    mca=widgets.fixed(mca), # valeur fixée au préalable, l'utilisateur n'a pas de choix
    guide = widgets.fixed(guide), # valeur fixée au préalable, l'utilisateur n'a pas de choix
    options=df_mca.columns, # Variable à choisir
    description='Variable:',
    value='BREATHING' # la valeur par défault
)

interactive_plot = widgets.interactive(ad.mca_plot_individuals_group, # la fonction prend 4 paramètres
                    variable=var_selector,
                    df_mca=widgets.fixed(df_mca),
                    mca=widgets.fixed(mca),
                    guide = widgets.fixed(guide))

display(interactive_plot)

In [ ]:
plot_modalites = ad.mca_plot_categories(df_mca, mca, 30)

Beaucoup de cases proche du bleu clair/bleu moyen illustrent des corrélations modestes à modérées, plutôt faibles globalement.
## ANALYSE A REFAIRE - LES VARIABLES ONT CHANGE

De petits îlots rouge‑orangés indiquent quelques paires assez corrélées positivement, par exemple autour de ```ACE11```(To the best of your knowledge, has this child EVER experienced any of the following? Treated or judged unfairly because of a health condition or disability) et ```K2Q60A```(Has a doctor, other health care provider, or educator EVER told you that this child has......Intellectual Disability (formerly known as Mental Retardation?), et plus bas autour de ```FOODSIT```(Which of these statements best describes your household's ability to afford the food you need DURING THE PAST 12 MONTHS?), ```ACE1```(SINCE THIS CHILD WAS BORN, how often has it been very hard to cover the basics, like food or housing, on your family's income?) et ```K8Q34```(DURING THE PAST MONTH, how often have you felt:...Angry with this child?).

In [ ]:
ad.heatmap_generator(df_mca)

## Enrichissement de la base de données

In [ ]:
# Enrichissement avec la jointure entre la base économique et la base géographique

df_eco_geo = cd.clean_enrichment_datasets(gdp, gdf)

Dans cette nouvelle base de données économiques, nous remarquons six colonnes vides pour les données de 2024. Le reste de la base ne présente aucune valeur manquante.

### Colonnes manquantes pour 2024
1. `Implicit regional price deflator`
2. `Real PCE (millions of constant (2017) dollars)`
3. `Regional price parities (RPPs)`
4. `Real per capita personal income`
5. `Real personal income (millions of constant (2017) dollars)`
6. `Real per capita PCE`

Suite à l'absence du déflateur des prix au niveau des états, certaines variables ne sont juste pas calculables. Cela arrive surtout dans les bases de données locales ou régionales, où certains indicateurs ne sont pas systématiquement rapportés.

### Colonnes présentes pour 2024
1. Disposable personal income
2. Personal consumption expenditures
3. Per capita personal income
4. Per capita personal consumption expenditures (PCE)
5. Per capita disposable personal income
6. Gross domestic product (GDP)
7. Real GDP (millions of chained 2017 dollars)
8. Personal income
9. Total employment (number of jobs)


In [ ]:
missing_values = df_eco_geo.isna().mean().sort_values(ascending=False) * 100
print(missing_values[:10])

# Création d'un indice synthétique annuel de santé au niveau des états

Afin de construire un indice synthétique de santé globale au niveau des états américains, nous allons adopter une méthodologie similaire à celle employée pour calculer l'Indice de Développement Humain. Nous allons calculer 3 sous-indicateurs pour chaque état : un indicateur maladies, un indicateur santé mentale et un indicateur économie-santé. Notre indice de santé globale sera calculée comme la moyenne géométrique de ces 3 indicateurs.

Nos sous-indicateurs seront une moyenne pondérée de ces variables. Afin de construire notre indicateur, notre idée initiale était de réaliser une ACP sur nos variables agrégées par état, et choisir comme
pondération les charges factorielles de chaque variable, i.e la contribution de chaque variable à la première composante de l'ACP. Ce choix est motivé par un désir de choisir une pondération issue des données afin d'éviter une pondération arbitraire. Cependant, comme nous le verrons plus bas, l'ACP ne donne pas de résultats satisfaisants (avec une première composant qui n'explique qu'un quart de la variance expliquée). Nous nous rabattons donc pour éviter de faire des choix plus arbitraires sur une moyenne simple des variables.

## Indicateur pour les maladies : illustration sur l'année 2023

Pour les maladies, nous avons volontairement choisi d'utiliser uniquement les questions sous la forme "Has a doctor or other health care provider EVER told you that this child has..." afin de simplifier l'étude.
Pour construire nos indicateurs, on va agréger ces variables par état grâce à la variable FIPSST.
Notre nouvelle base de données sera une base où les individus seront chaque état, avec les variables ci-dessus en moyenne.
Pour construire un indicateur pertinent (plus l'indicateur est grand, plus la population est saine), on inversera le sens des catégories pour General Health et Teeth description (donc 1 = Poor à 5 = Excellent pour General Health, 0 = Pas de dents à 5 = Excellent pour Teeth description). De même, pour une question de lisibilité, on a remplacé pour les variables binaires 1/2 par 0/1, ou 0 = a la maladie, 1 = ne l'a pas.

In [ ]:
df_2021, df_2022, df_2023, df_2024 = (
    dfs_final[str(year)] for year in range(2021, 2025)
)

groups = {"FIPSST", "FWC"}
health_total_vars = list(set(health_vars) | set(groups))

# étape n°1 : Inversion de l'échelle
df_sickness_2023 = df_2023[health_total_vars].copy()
df_sickness_2023[health_category_vars] = 6 - df_sickness_2023[health_category_vars]

# étape n°2 : Inversion des variables binaires
# Transformation : sain = 1, malade = 0
for var in health_bin_vars:
    df_sickness_2023[var] = df_sickness_2023[var] - 1

In [ ]:
# Agrégation par état
def weighted_mean(x, w):
    return (x * w).sum() / w.sum()

df_sickness_state_2023 = df_sickness_2023.groupby("FIPSST").apply(
    lambda g: pd.Series({var: weighted_mean(g[var], g["FWC"]) for var in health_vars}))

In [ ]:
X = df_sickness_state_2023[health_vars]

In [ ]:
scaler = StandardScaler()
X_std = scaler.fit_transform(X)
X_std = pd.DataFrame(X_std, columns=health_vars, index=df_sickness_state_2023.index)

In [ ]:
pca = PCA()
pca.fit(X_std)

explained_variance = pca.explained_variance_ratio_
print("Variance expliquée par PC1 :", explained_variance[0])

Nous obtenons une première composante expliquant une part de la variance assez faible, seulement 22% (il aura fallu au moins 4 composantes pour expliquer plus de 50% de la variance). La pondération que nous pouvons en extraire ne présentera donc pas d'avantages comparé à une pondération arbitraire comme une pondération uniforme. Nous choisissons donc une pondération uniforme pour notre indicateur.

In [ ]:
plt.plot(range(1, len(explained_variance)+1), explained_variance.cumsum(), marker='o')
plt.xlabel("Composante")
plt.ylabel("Variance cumulée")
plt.show()

In [ ]:
df_sickness_state_2023["sub_indicator_health_2023"] = X.mean(axis=1)

In [ ]:
df_sickness_state_2023["sub_indicator_health_2023"].describe()

Notre indicateur peut prendre des valeurs comprises entre 1/15 = 0.066 (cas où toutes les variables sont au minimum) et 23/15 = 1.533 (en l'occurence ici, l'indicateur est compris entre 1.3 et 1.42 pour les états américains). Nous effectuons une transformation affine, pour le ramener entre 0 et 1 (de façon similaire aux sous-indices utilisés pour le calcul de l'indice de développement humain).

In [ ]:
I = df_sickness_state_2023["sub_indicator_health_2023"]

df_sickness_state_2023["sub_indicator_health_2023"] = (I-1/15)/(23/15 - 1/15)
df_sickness_state_2023["sub_indicator_health_2023"]

I.describe()

## Indicateur pour la santé mentale : illustration sur l'année 2023

In [ ]:
mental_heath_indicator_2023 = model.calculate_indicator("2023", dfs_final, "mental_health", 
                                             mental_category_vars, mental_bin_vars,
                                             groups)

In [ ]:
mental_heath_indicator_2023.describe()

## Indicateur santé-économie : illustration pour l'année 2023

In [ ]:
df_eco_geo_indic = cd.clean_eco_data(df_eco_geo)

Nous travaillons en premier lieu sur les 4 variables du survey NSCH, afin de les agréger à l'échelle de l'état, pour ensuite faire la jointure avec les données macroéconomiques.

De nouveau, on change CURRINS en une variable binaire 0/1, de telle sens que 1 = Yes, comme c'est une issue positive (et comme notre indicateur sera une moyenne des variables, donc une fonction croissante en les variables, on veut bien que plus une variable représente une issue, positive au niveau santé, plus l'indicateur est haut). De même pour les variables catégorielles.

In [ ]:
NSCH_eco_vars = NSCH_eco_bin_vars + NSCH_eco_cat_vars
NSCH_eco_total_vars = ["FIPSST", "FWC"] + NSCH_eco_vars

df_health_eco_2023 = df_2023[NSCH_eco_total_vars]
df_health_eco_2023[NSCH_eco_cat_vars] = 5 - df_health_eco_2023[NSCH_eco_cat_vars]
#CURRINS = abs(CURRINS - 2), de telle sorte que 2 soit envoyé sur 0, et 1 va rester à 1
df_health_eco_2023["CURRINS"] = abs(df_health_eco_2023["CURRINS"] - 2)

df_health_eco_state_2023 = df_health_eco_2023.groupby("FIPSST").apply(lambda g: pd.Series({var: weighted_mean(g[var], g["FWC"]) for var in NSCH_eco_vars}))

De plus, ici, on a 9 variables macroéconomiques contre 4 variables "microéconomiques" liées à la santé, l'assurance et au fait de pouvoir subvenir au besoin de l'enfant.
Pour éviter une prédominance des variables macroéconomiques dans la moyenne, on réalise d'abord la moyenne sur les variables du NSCH d'une part, et d'autre part, la moyenne sur les variables macros, puis on construira notre indicateur comme moyenne de ces deux moyennes.

In [ ]:
X = df_health_eco_state_2023[NSCH_eco_vars]
df_health_eco_state_2023["sub_indicator_microeco_2023"] = X.mean(axis=1)

I = df_health_eco_state_2023["sub_indicator_microeco_2023"]
df_health_eco_state_2023["sub_indicator_microeco_2023"] = (I-2/3)/(3 - 2/3)

In [ ]:
eco_templates = [
    "Disposable personal income",
    "Gross domestic product (GDP)",
    "Per capita disposable personal income 7/",
    "Per capita personal consumption expenditures (PCE) 8/",
    "Per capita personal income 6/",
    "Personal consumption expenditures",
    "Personal income",
    "Real GDP (millions of chained 2017 dollars) 1/",
    "Total employment (number of jobs)"
]


state_eco_vars_2021 = [f"{2021}_{var}" for var in eco_templates]
state_eco_vars_2022 = [f"{2022}_{var}" for var in eco_templates]
state_eco_vars_2023 = [f"{2023}_{var}" for var in eco_templates]
state_eco_vars_2024 = [f"{2024}_{var}" for var in eco_templates]



In [ ]:
# On peut maintenant faire la même êtude sur les variables macroéconomiques
df_macro_state_2023 = df_eco_geo_indic[["FIPSST"] + state_eco_vars_2023]
X = df_macro_state_2023[state_eco_vars_2023]

scaler = StandardScaler()
X_std = scaler.fit_transform(X)
X_std = pd.DataFrame(X_std, columns=state_eco_vars_2023, index=df_macro_state_2023.index)

pca = PCA()
pca.fit(X_std)

explained_variance = pca.explained_variance_ratio_
print("Variance expliquée par PC1 :", explained_variance[0])

In [ ]:
loadings = pca.components_[0]
weights_raw = abs(loadings)
weights = weights_raw / weights_raw.sum()
df_macro_state_2023["sub_indicator_macroeco_2023"] = (X_std * weights).sum(axis=1)

#Afin de se ramener à des valeurs entre 0 et 1, on normalise notre indicateur et on effectue la transformation I <- F(I) où F est la fonction de répartition de la loi normale standard.
std_indic = (
    df_macro_state_2023["sub_indicator_macroeco_2023"] - df_macro_state_2023["sub_indicator_macroeco_2023"].mean()
) / df_macro_state_2023["sub_indicator_macroeco_2023"].std()

df_macro_state_2023["sub_indicator_macroeco_2023"] = norm.cdf(std_indic)

On calcule enfin le sous-indicateur santé-économie final comme moyenne des deux.

In [ ]:
df_macro_state_2023["FIPSST"] = pd.to_numeric(df_macro_state_2023["FIPSST"], errors="coerce")

df_eco_2023 = df_macro_state_2023.merge(df_health_eco_state_2023, on="FIPSST", how="inner")
df_eco_2023.set_index("FIPSST", inplace=True)
df_eco_2023["sub_indicator_eco_2023"] = (df_eco_2023["sub_indicator_macroeco_2023"] + df_eco_2023["sub_indicator_microeco_2023"])/2

### Indicateur final pour l'année 2023

In [ ]:
mental_heath_indicator_2023 = mental_heath_indicator_2023.to_frame(name="sub_indicator_mental_2023")
mental_2023 = mental_heath_indicator_2023[["sub_indicator_mental_2023"]]
health_2023 = df_sickness_state_2023[["sub_indicator_health_2023"]]
eco_2023 = df_eco_2023[["sub_indicator_eco_2023"]]

df_final = (
    mental_2023
    .join(health_2023, how="inner")
    .join(eco_2023, how="inner")
)

df_final["indicator_global_health_2023"] = (df_final["sub_indicator_mental_2023"]* df_final["sub_indicator_health_2023"] * df_final["sub_indicator_eco_2023"])**(1/3)

In [ ]:
df_final["indicator_global_health_2023"].describe()

In [ ]:
df_final["indicator_global_health_2023"].sort_values()